<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/zeroshot_%EC%82%B0%EC%88%A0%ED%8F%89%EA%B7%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm
import pandas as pd

# 첫 번째 모델
zeroshot_model30000 = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/프로젝트/final project/data/zero_model_epoch_2").to('cuda')

# 두 번째 모델
model_name = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to('cuda')
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer, device=0)

# 공통 토크나이저
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

# 데이터 전처리
inputs = tokenizer(reviews_to_predict, truncation=True, padding=True, return_tensors="pt").to('cuda')
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'])
data_loader = DataLoader(dataset, batch_size=16, shuffle=False)

# 모델 1 예측
scores_from_model1 = []
# classifier pipeline 미사용
with torch.no_grad():
    for batch in tqdm(data_loader):
        input_ids, attention_mask = (item.to('cuda') for item in batch)
        outputs = zeroshot_model30000(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()

        for prob in probs:
            scores = {candidate_labels[idx]: f"{val:.2f}" for idx, val in enumerate(prob)}
            scores_from_model1.append(scores)

# 모델 2 예측
scores_from_model2 = []
# classifier pipeline 사용
for review in tqdm(reviews_to_predict):
    output = classifier(review, candidate_labels, multi_label=True)
    scores = {label: round(score, 2) for label, score in zip(output['labels'], output['scores'])}
    scores_from_model2.append(scores)

averaged_scores_list = []
predicted_labels_list = []
.
for score1, score2 in zip(scores_from_model1, scores_from_model2):

    averaged_scores = {}
    predicted_labels = []

    # 각 레이블에 대한 두 모델의 점수를 순회합니다.
    for label in score1.keys():
        # 두 모델의 점수를 산술평균
        averaged_score = (float(score1[label]) + float(score2[label])) / 2
        averaged_scores[label] = round(averaged_score, 2)

        # 산술평균 점수가 0.5 이상이면 해당 레이블 추가
        if averaged_score > 0.5:
            predicted_labels.append(label)

    # 각 리뷰에 대한 산술평균 점수를 averaged_scores에 추가.
    averaged_scores_list.append(", ".join([f"{k}: {v}" for k, v in averaged_scores.items()]))
    # 각 리뷰에 대한 산술평균 점수가 0.5 이상인 레이블을 predicted_labels에 추가
    predicted_labels_list.append(", ".join(predicted_labels))

result_df = pd.DataFrame({
    'Review': reviews_to_predict,
    'Predicted_Labels': predicted_labels_list,
    'Averaged_Label_Scores': averaged_scores_list
})

print(result_df)
